In [1]:
# ! python -m site
### installing package in jupyter notebook
# import sys # replace the path below with the location of your .whl file :
#!{sys.executable} -m pip install C:\Users\BuruzsA\PycharmProjects\flows1d\dist\flows1d-1.0.0b4-py3-none-any.whl
## if you want to uninstall for test/updates 

In [2]:
import logging
import sys
logging.basicConfig(stream = sys.stdout, level = logging.INFO)

In [3]:
#sys.path.append("C:/Users/BuruzsA/PycharmProjects/")
sys.path.append("C:/Users/BuruzsA/PycharmProjects/flows1d") ## this is not needed, if the package is installed in jupyter
from flows1d.core import nozzleFactory, NozzleParams, nozzleSolver, EjectorGeom, refProp, EjectorMixer

In [4]:
import numpy as np
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import time
import importlib
import math

%matplotlib notebook

from IPython.display import Image
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))
from time import time
t0 = time()

In [5]:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', 140)
np.core.arrayprint._line_width = 180
np.set_printoptions(linewidth= 180)

In [6]:
# Schlemminger Hochtemperatur Wärmepumpe mit Ejektor
Rin = 1.94/2.0
Rt = 0.30688/ 2.0 
Rout = 0.68568 / 2.0
gamma_conv = 15.7/2.0 # grad
gamma_div = 6/2.0 # grad
Lcon = (Rin-Rt) / math.tan(gamma_conv * math.pi/180)
print(f"Primary nozzle: inlet Radius {Rin} cm,\n Throat radius {Rt} cm \n convergent lenght {Lcon} cm")
Ldiv = (Rout-Rt) / math.tan(gamma_div * math.pi/180)
print(f"Primary nozzle: Outlet Radius {Rout} cm,\n divergent length {Ldiv}")
nozzle = nozzleFactory.ConicConic(Rin = Rin, Lcon=Lcon, Rt = Rt, Ldiv = Ldiv, Rout = Rout)
nozzle.setFriction( 1.0e-2 )
ejector = EjectorGeom(nozzle, Dm = 2.0 )
mixstart = Lcon + Ldiv + 3.1769
gamma_diffusor = 2.5/2.0 ## or the half of it??
diffuserLen = (42 - 20)/ 10/ 2 / math.tan(gamma_diffusor * math.pi/180)
print(f"mixer start {mixstart} cm, diffuser length {round(diffuserLen,3)} cm")
ejector.setMixer(mixerstart=mixstart, mixerLen=11.2, diffuserLen=diffuserLen, diffuserHeight=2.10)
ejectorPlot = ejector.draw()

Primary nozzle: inlet Radius 0.97 cm,
 Throat radius 0.15344 cm 
 convergent lenght 5.922590285032108 cm
Primary nozzle: Outlet Radius 0.34284 cm,
 divergent length 3.6139672886557235
mixer start 12.713457573687831 cm, diffuser length 50.412 cm


<IPython.core.display.Javascript object>

In [7]:
mixstart

12.713457573687831

In [8]:

## case A
pin = 1800.0  # kPa  2000
Tin =  400   # Kelvin 127 C
Tsuc = 317.25 #  suction temperature K -> 44C
Psuc =  150 # suction pressure in kPa
## case B:
# pin = 2139.0  # kPa  2000
# Tin =  409.32   # Kelvin  400
# Tsuc = 336.63 #  suction temperature K
# Psuc =  430 # suction pressure in kPa

fluid = "R1233zd"
RP = refProp.setup(fluid)
[Din, hin] = refProp.getDh_from_TP(RP, Tin, pin)

In [9]:
matinfo = RP.INFOdll(1)
matinfo

INFOdlloutput(wmm=130.4961896, Ttrp=195.15, Tnbpt=291.47, Tc=438.75, Pc=3572.6, Dc=3.67, Zc=0.2668495808966737, acf=0.305, dip=1.44, Rgas=8.314472)

In [10]:
matinfo.Tc - 273.15 # critical temp. in Celsius

165.60000000000002

In [11]:
nsolver = nozzleSolver.NozzleSolver(nozzle, fluid, 1, solver="AdamAdaptive", mode="basic")
nsolver.setFriction(1e-2)
vin_crit = nsolver.calcCriticalSpeed(pin, hin, 0.1, maxdev= 1e-3, chokePos="divergent_part" )

friction of the flow calc set to 0.01
{'massflow': 0.08351844057806011}
INFO:root: x interval end reached xlast 9.460068807739248 +  dX 0.1 vs xend 9.536557573687832
INFO:root:solver finished in 212 iterations. final point xlast =9.536557573687832
{'massflow': 0.16703688115612023}
3 dimensional Newton Raphson has not converged in 100 iterations
error = 1.2374033881850905
INFO:root:Newton root finding failed at 5.622950896627165 + 0.00036371248866878337
3 dimensional Newton Raphson has not converged in 100 iterations
error = 0.46169630120023886
INFO:root:Newton root finding failed at 5.6231363899963815 + 3.6371248866878337e-06
3 dimensional Newton Raphson has not converged in 100 iterations
error = 0.002540562739441403
INFO:root:Newton root finding failed at 5.623136426367631 + 2.7396129294706037e-06
INFO:root:update failed by 5.623136426367631 with dX=2.7396129294706037e-06. stop solver
result is between 0.3 and 0.6
{'massflow': 0.12527766086709016}
3 dimensional Newton Raphson has not

The choked flow in the ejector motive nozzle:

In [12]:
print(f"calculated critical choking inlet speed = {round(vin_crit,5)} m/s")
mass_flow_crit = vin_crit * refProp.getTD(nsolver.RP, hin, pin)['D'] * nsolver.nozzle.Aprofile(0)*1e-4
print(f"critical mass flow is {round(mass_flow_crit,5)} kg/sec")

calculated critical choking inlet speed = 0.37207 m/s
critical mass flow is 0.10358 kg/sec


## Compared to the CFD result of critical mass flow of 0.108 kg/sec !!!

In [13]:
nozzle_crit0 = nsolver.solveNplot(vin_crit, pin, hin)

{'massflow': 0.10358244095130502}
INFO:root: x interval end reached xlast 9.534282185981754 +  dX 0.1 vs xend 9.536557573687832
INFO:root:solver finished in 1143 iterations. final point xlast =9.536557573687832
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


<IPython.core.display.Javascript object>

ups, by the critical speed, the flow still stays subsonic! So let's apply a small kick in the throat:

In [14]:
res_crit= None

In [15]:
print("############# resolve with kicks ############ ")
v01 = vin_crit #-0.004
sol_1 = nsolver.solveAdaptive1DBasic(v01, pin, hin, 0.0, nsolver.nozzle.xt)
vph_throat = sol_1.iloc[-1]
v = vph_throat["v"]
p = vph_throat["p"]
h = vph_throat["h"]
dv_kick = 3.0
dp_kick = nsolver.pFromV_MassConst(v = vph_throat["v"], dv = dv_kick, p = vph_throat["p"], h = vph_throat["h"])
print(f"mass conserving artificial kick: dv = {dv_kick} m/s, dp = {dp_kick} kPa")

############# resolve with kicks ############ 
{'massflow': 0.10358244095130502}
INFO:root: x interval end reached xlast 5.922518197582624 +  dX 0.00032369675117749043 vs xend 5.922590285032108
INFO:root:solver finished in 104 iterations. final point xlast =5.922590285032108
mass conserving artificial kick: dv = 3.0 m/s, dp = -40.5037644207495 kPa


In [16]:
res_crit = nsolver.solveKickedNozzle(v01, pin , hin, kicks = {'v': dv_kick, 'p': -dp_kick},
                                     solver= "adaptive_implicit", step0 = 0.001, maxStep = 0.005)
print(f"throat by {nsolver.nozzle.xt}")
nsolver.plotsol(res_crit, title = f"choked nozzle with friction = {nsolver.frictionCoef}.\n with artifical kick by throat with {dv_kick} m/sec ")
print(res_crit.tail(1))

{'massflow': 0.10358244095130502}
INFO:root: x interval end reached xlast 5.922518197582624 +  dX 0.00032369675117749043 vs xend 5.922590285032108
INFO:root:solver finished in 104 iterations. final point xlast =5.922590285032108
{'massflow': 0.10358244054723216}
INFO:root: x interval end reached xlast 9.534590285032538 +  dX 0.005 vs xend 9.536557573687832
INFO:root:solver finished in 2954 iterations. final point xlast =9.536557573687832
throat by 5.922590285032108


<IPython.core.display.Javascript object>

             x           v           p          h          d           c      mach  quality
3056  9.536558  176.856175  173.285523  352.44204  15.861091  106.865549  1.654941  0.59374


In [17]:
res_crit[140:150]

,x,v,p,h,d,c,mach,quality
140,5.95959,44.882380,1299.023794,367.073879,304.281293,69.805562,0.642963,0.176963
141,5.96059,45.026380,1297.019046,367.067406,303.103662,69.842057,0.644689,0.177632
142,5.96159,45.168923,1295.035405,367.060977,301.943483,69.878414,0.646393,0.178293
143,5.96259,45.310064,1293.072097,367.054592,300.800115,69.914635,0.648077,0.178948
144,5.96359,45.449855,1291.128396,367.048248,299.672957,69.950726,0.649741,0.179596
145,5.96459,45.588344,1289.203617,367.041945,298.561442,69.986690,0.651386,0.180238
146,5.96559,45.725576,1287.297119,367.035679,297.465037,70.022529,0.653012,0.180873
147,5.96659,45.861594,1285.408293,367.029450,296.383237,70.058248,0.654621,0.181502
148,5.96759,45.996440,1283.536567,367.023257,295.315567,70.093849,0.656212,0.182126
149,5.96859,46.130151,1281.681399,367.017098,294.261577,70.129335,0.657787,0.182743


In [18]:
print(res_crit.shape)
res_crit.tail(5)

(3057, 8)


,x,v,p,h,d,c,mach,quality
3052,9.519590,176.563617,174.353928,352.493738,15.970108,106.818184,1.652936,0.593114
3053,9.524590,176.649977,174.038083,352.478486,15.937864,106.832183,1.653528,0.593299
3054,9.529590,176.736221,173.723069,352.463247,15.905719,106.846147,1.654119,0.593483
3055,9.534590,176.822348,173.408884,352.448022,15.873671,106.860077,1.654709,0.593668
3056,9.536558,176.856175,173.285523,352.442040,15.861091,106.865549,1.654941,0.593740


In [19]:
nsolver.plotsol(res_crit, title = "choked nozzle with friction = {} ".format(nsolver.frictionCoef))

<IPython.core.display.Javascript object>

that looks better ... 

## Mixing calculations

#### set up the mixer properties. 

In [20]:
mixingParams = { 'massExchangeFactor' : 2.e-4, 'dragFactor': 0.01, 'frictionInterface': 0.0, 'frictionWall': 0.0015}
mixer = EjectorMixer.EjectorMixer(fluid, ejector, mixingParams)

In [21]:
mixer.setSuctionMassFlow(None)
mixer.setSingleChoke(True)
mixer.ejector.Asi = 4 # cm2 of the suction nozzle inlet.

# calculate the Pre-mixing chamber 

In [22]:
mixerin = mixer.premixWrapSolve(res_crit, Psuc ,Tsuc)

critical flow nozzle outlet quantities
               3056
x          9.536558
v        176.856175
p        173.285523
h        352.442040
d         15.861091
c        106.865549
mach       1.654941
quality    0.593740
INFO:root:calculation of single-chocking subcritical mode incl. suction mass flow rate calculations
{'Tsi': 317.25, 'psi': 150, 'massFlowPrim': 103.58244054659919, 'ho': 352.442039678156, 'vo': 176.85617470661478, 'so': 1.5049049205926364, 'Am': 3.141592653589793}
INFO:root:pre-mix calculations with single choking and suction mass flow rate calculation
at Y point after pre-mixing
{'py': 150.00000000000006, 'vpy': 185.60942088637736, 'hpy': 350.85566438319614, 'Dpy': 13.566044777386496, 'Apy': 0.4113702470023567, 'vsy': 2.0575097643243165e-08, 'hsy': 438.21725212974235, 'Dsy': 7.790023150229453, 'Asy': 2.7302224065874365, 'massFlowSecond': 4.3760137643100484e-08}
Pressure[kPa] suction 150 . primary nozzle exit : 173.286 
equalized pressure after pre-mixing : 150.0 kPa
cro

In [26]:
mixerin

{'py': 150.00000000000006,
 'vpy': 185.60942088637736,
 'hpy': 350.85566438319614,
 'Dpy': 13.566044777386496,
 'Apy': 0.4113702470023567,
 'vsy': 2.0575097643243165e-08,
 'hsy': 438.21725212974235,
 'Dsy': 7.790023150229453,
 'Asy': 2.7302224065874365,
 'massFlowSecond': 4.3760137643100484e-08,
 'massFlowPrim': 103.58244054659919}

the premixing output is the input for the mixer

In [25]:
mixerinput = [ mixerin[key] for key in [  'py', 'vpy', 'vsy', 'hpy', 'hsy', 'Apy', 'Asy']]

solve the initial value ODE:

In [24]:
solMix = mixer.solveMix(mixerinput)

ValueError: math domain error

The 2 annular flows: p: primary (motive flow), and s for the secondary ( suction flow) 

In [ ]:
print(solMix)

In [ ]:
mixer.plotSolution(res_crit, solMix, "no shock in mixer",  ejectorPlot)
solNoShock = solMix

In [ ]:
print(f"elapsed time {round(time()-t0, 3)} sec")

In [ ]:
#P_CFD = pd.read_csv("C:/Users/BuruzsA/Documents/projects/ETHP/ManuelCFD/pressure_CFD1.csv")
P_CFD = pd.read_csv("C:/Users/BuruzsA/Documents/projects/ETHP/ManuelCFD/p_stat_CFD.csv")
plt.figure()
plt.plot(P_CFD.iloc[:,0]*100,P_CFD.iloc[:,1]*1e-3 )
plt.plot(res_crit['x'], res_crit['p'], color = '#22AA22')
plt.plot(solMix['x'], solMix['p'], color = '#22AA22')
plt.plot()
plt.ylabel("p[kPa]")
plt.legend(["CFD", "1D Simulation", "1D Simulation"])
plt.ylim((0, res_crit['p'][0]*1.1))

## Diffusor outlet pressure vs suction mass flow rate

In [ ]:
# try various mass flow rates in g/sec:
res = []
for suctionMassFlow in np.arange(10, 200, 10):
    mixingParams = { 'massExchangeFactor' : 2.e-4, 'dragFactor': 0.01, 'frictionInterface': 0.0, 'frictionWall': 0.0015}
    mixer = EjectorMixer.EjectorMixer(fluid, ejector, mixingParams)
    mixer.setSuctionMassFlow(suctionMassFlow)
    mixerin = mixer.premixWrapSolve(res_crit, Psuc ,Tsuc)
    mixerinput = [ mixerin[key] for key in [  'py', 'vpy', 'vsy', 'hpy', 'hsy', 'Apy', 'Asy']]
    solMix = mixer.solveMix(mixerinput)
    pout = round(solMix['p'].iloc[-1],3)
    print(f"suction massflow: {suctionMassFlow}. pressure outlet : {pout}")
    res.append([suctionMassFlow, pout])
mdp = pd.DataFrame(res, columns = ["massFlowRate[g/s]","pressure[kPa]"])

In [ ]:
mdp

# APPENDIX

without shock wave both flow will be supersonic in the mixer and also in the diffuser. 
This gives us too low exit pressures. So let's assume that there is a shockwave at some x_sh, and afterwards the both flows are mixed:

In [ ]:
x_sh= 13.5 # let's say there is a normal shock wave at x_sh!

In [ ]:
solNoShock # this would be the solution without shock:

now we calculate, how the solution looks like with a shock wave in x_sh:

In [ ]:
beforeShock, afterShock, solMixShocked = mixer.mixingShock(x_sh, solNoShock, mergeflows = True)

let's plot the results:

In [ ]:
mixer.plotSolution(res_crit, solMixShocked, "normal shock at {} cm".format(x_sh),  ejectorPlot)

so the shock at x_sh kicks the flow down to subsonic region.
the values at the diffuser exit:

In [ ]:
solMixShocked.iloc[-1]

 x in cm, p in kPa, vp/vs = primary/secondary speed in m/s, hp, hs in kJ/kg/K, Ap,As in cm^2 

In [ ]:
solMixShocked

In [ ]:
solMixShocked['Dp'] = [refProp.getTD(nsolver.RP, solMixShocked['hp'][i] ,solMixShocked['p'][i] )['D'] for i in range(solMixShocked.__len__()) ]
solMixShocked['Ds'] = [refProp.getTD(nsolver.RP, solMixShocked['hs'][i] ,solMixShocked['p'][i] )['D'] for i in range(solMixShocked.__len__()) ]


Let's check the mass flow rates!

In [ ]:
MFp = solMixShocked['Dp'] * solMixShocked['vp'] *  solMixShocked['Ap']* 1e-4 # primary mass flow rate
MFs = solMixShocked['Ds'] * solMixShocked['vs'] *  solMixShocked['As']* 1e-4 # secondary mass flow rate
plt.figure()
plt.plot(solMixShocked['x'], MFp+MFs)
plt.axvline(x=x_sh, linestyle=':', color='b')
plt.ylabel("mass flow [kg/sec]")
plt.ylim(ymin = min(MFp+MFs)*0.95, ymax = max(MFp+MFs)*1.05)

Cheking if the total mass flow rate really constant:

ok, that looks good.

In [ ]:
print("pressure at the diffuser exit = {} kPa".format(solMixShocked.iloc[-1]['p']))

TODO: find the x_sh shock location in dependence of the exit pressure (= invert the x_sh -> p_{diffuser exit} function)

In [ ]:
MFp[0]

In [ ]:
Pdiffout = solMixShocked.iloc[-1]['p']
Pdiffout

In [ ]:
efficiency = mixer.calcEfficiency(pin, Tin, Psuc, Tsuc, Pdiffout, MFp[0] , MFs[0] )

Ejektor Efficiency according to Elbel

In [ ]:
efficiency

## Flow simulation from outlet pressure ##
Let's see the more realistic inverse problem: Usually we can not measure the shock-wave position, but we can measure pretty accurately the **pressure in the diffusor exit**. We can even set the diffusor exit-pressure, and thus it's one of the key control variables when we set up an ejector.  
So our python package provides a function to calculate the double choking case shock-wave position based on the exit pressure of the diffusor:

In [ ]:
P_outlet = 230 # kPa for example
shockFromP = mixer.getChokePosition(P_outlet, solNoShock)

In [ ]:
shockFromP

In [ ]:
#beforeShock, afterShock, solMixShockedP = mixer.mixingShock(shockFromP['shockPos'], solNoShock, mergeflows = True)

In [ ]:
#mixer.plotSolution(res_crit, solMixShockedP, "for outlet Pressure {} kPa \n calculated normal shock at {} cm".format(P_outlet, round(shockFromP['shockPos'],3) ),  ejectorPlot)